In [2]:
import pandas as pd
import numpy as np
import datetime   # datetime 내 timedelta는 몇주전후 까지만 나와 있음
from dateutil.relativedelta import relativedelta # 몇달전후, 몇년전후를 구하기 위함
from tqdm import tqdm
from scipy import stats

In [3]:
import weatherAPI 

In [4]:
other=pd.read_csv("month_other_weather.csv", encoding='utf-8-sig')

In [5]:
other.head()

,month,시도,avg_temp(C),daily_rain(mm),avg_wind(m/s),avg_r_humidity(%)
0,1,강원도,-4.4,0.2,1.4,60.0
1,1,경기도,-3.3,0.3,1.6,58.9
2,1,경상남도,0.1,0.4,1.6,54.5
3,1,경상북도,-0.3,0.6,2.3,50.7
4,1,부산광역시,3.1,0.5,3.3,41.8


In [6]:
occur=pd.read_csv("month_occur_weather.csv", encoding='utf-8-sig')

In [7]:
occur.head()

,month,시도,avg_temp(C),daily_rain(mm),avg_wind(m/s),avg_r_humidity(%),cnt
0,1,강원도,2.6,0.0,1.4,62.5,2
1,1,경기도,-3.4,0.6,1.6,63.2,66
2,1,경상남도,-1.0,0.0,1.8,54.0,12
3,1,경상북도,-0.9,0.1,1.7,50.2,3
4,1,부산광역시,6.1,0.0,1.1,58.0,2


#### RISK TABLE SET 제작
    1. occur과 non_occur과의 관계성이 나와있는 것만으로 table을 제작함
    2. 여름 5-9월 까지는 상대적으로 낮은 온도에서 발생하는 경향이 있음. (7~9월은 data 부족으로 판단)
       반면에 겨울에는 겨울 평상시 온도보다 약간 더 높은 온도에서 발생하는 경향이 있음 (12~2월) 
    3. 여름 5-9월 까지는 상대적으로 낮은 강수량일 경우 발생하는 경향이 있음. (7~9월은 data 부족으로 판단)
    4. 즉 12~2월은 온도가 겨울 평균온도보다 차이별로 높을경우 적용, 5~9월은 여름 평균온도보다 차이별로 낮을경우 적용

In [29]:
# 예시
occur.query("month==2")['avg_temp(C)'].mean()

3.227272727272728

In [30]:
other.query("month==2")['avg_temp(C)'].mean()

1.4363636363636363

In [37]:
month=[1,2,5,6,7,9,12]
cols_name=['month','avg_temp(C)','daily_rain(mm)','risk']
data=pd.DataFrame(columns=cols_name)
for mon in month:
    dic={}
    dic['month']=[mon,mon]
    avg_temp=np.round(other.query("month==@mon")['avg_temp(C)'].mean(),2)
    if mon in [5,6,7,9]:
        avg_rain=np.round(other.query("month==@mon")['daily_rain(mm)'].mean(),2)
        diff_temp=np.round(occur.query("month==@mon")['avg_temp(C)'].mean(),2)
        diff_rain=np.round(occur.query("month==@mon")['daily_rain(mm)'].mean(),2)
        dic['avg_temp(C)']=[avg_temp,diff_temp]
        dic['daily_rain(mm)']=[avg_rain, diff_rain]
    else:
        avg_rain=np.nan
        diff_temp=np.round(occur.query("month==@mon")['avg_temp(C)'].mean(),2)
        dic['avg_temp(C)']=[avg_temp,diff_temp]
        dic['daily_rain(mm)']=[avg_rain,avg_rain]
    dic['risk']=[0,10]
    temp=pd.DataFrame.from_dict(dic)
    data=pd.concat([data, temp], axis=0)
    data.reset_index(inplace=True, drop=True)

In [39]:
data.to_csv('Weather_Risk_table_ver2.csv', encoding='utf-8-sig', index=False)

In [41]:
data

,month,avg_temp(C),daily_rain(mm),risk
0,1,-1.07,NaN,0
1,1,0.48,NaN,10
2,2,1.44,NaN,0
3,2,3.23,NaN,10
4,5,18.56,2.66,0
5,5,16.61,3.00,10
6,6,22.32,4.52,0
7,6,20.79,1.32,10
8,7,25.10,10.40,0
9,7,27.20,5.10,10


In [12]:
occur.query("month==1").sort_values(['cnt'])

,month,시도,avg_temp(C),daily_rain(mm),avg_wind(m/s),avg_r_humidity(%),cnt
0,1,강원도,2.6,0.0,1.4,62.5,2
4,1,부산광역시,6.1,0.0,1.1,58.0,2
3,1,경상북도,-0.9,0.1,1.7,50.2,3
8,1,충청북도,-0.5,0.6,1.2,58.4,5
2,1,경상남도,-1.0,0.0,1.8,54.0,12
7,1,충청남도,-1.3,0.1,1.3,72.6,25
6,1,전라북도,1.0,1.0,1.8,72.9,52
5,1,전라남도,1.7,1.0,2.8,69.0,58
1,1,경기도,-3.4,0.6,1.6,63.2,66


In [16]:
other.query("month==1")

,month,시도,avg_temp(C),daily_rain(mm),avg_wind(m/s),avg_r_humidity(%)
0,1,강원도,-4.4,0.2,1.4,60.0
1,1,경기도,-3.3,0.3,1.6,58.9
2,1,경상남도,0.1,0.4,1.6,54.5
3,1,경상북도,-0.3,0.6,2.3,50.7
4,1,부산광역시,3.1,0.5,3.3,41.8
5,1,전라남도,0.9,0.5,3.0,64.5
6,1,전라북도,-1.0,0.6,1.5,68.6
7,1,충청남도,-1.9,0.5,1.9,67.1
8,1,충청북도,-2.8,0.5,1.4,60.0


In [17]:
hpai=pd.read_csv('HPAI_Weather_occur.csv', encoding='utf-8-sig')

In [22]:
hpai=hpai.drop(['축종', '주소', '진단일', '사육수', '지역명', 'location', 'date', 'lowest_temp(C)',
                  'highest_temp(C)', 'maxWsWd(16방위)'], axis=1) 

In [23]:
hpai['발생일']=hpai['발생일'].astype('str')

In [24]:
hpai['발생일']=hpai['발생일'].map(lambda x : datetime.datetime.strptime(x, '%Y%m%d'))

In [34]:
hpai['month']=hpai['발생일'].map(lambda x: x.month)
hpai['year']=hpai['발생일'].map(lambda x: x.year)
hpai['day']=hpai['발생일'].map(lambda x: x.day)

In [51]:
jgg=hpai.query('시도=="전라북도"').groupby(['year','month','day']).mean().query("month==7")[['avg_temp(C)']]

In [52]:
jgg

,,,avg_temp(C)
year,month,day,


In [50]:
jgg.describe() # -8.23075< x < 10.81875 0 9 10 9 (구간을 범위로 나눴다.) 안되면 5점

,avg_temp(C)
count,6.000000
mean,19.348889
std,1.178373
min,17.700000
25%,18.616667
50%,19.433333
75%,20.070000
max,20.900000


In [44]:
from scipy import stats

In [45]:
test_stat, p_value = stats.shapiro(jgg)
print("test_stat: %f, pvalue=%f"%(test_stat, p_value)) # 비정규성.. 

test_stat: 0.883596, pvalue=0.009825


In [72]:
distance=hpai[['year','month','day','시도','시군구']]

In [73]:
distance['cnt']=1

<ipython-input-73-ef07ac554c7e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  distance['cnt']=1


In [74]:
distance=distance.groupby(['시도','시군구','month']).sum()

In [80]:
sumit=distance.sort_values(['month','cnt']).reset_index()

In [132]:
sumit.head()

,시도,시군구,month,year,day,cnt
0,강원도,인제군,1,2017,3,1
1,강원도,횡성군,1,2017,6,1
2,경기도,광주시,1,2015,19,1
3,경기도,남양주시,1,2021,11,1
4,경기도,연천군,1,2017,12,1


In [118]:
occur_D=pd.read_csv('발생지간거리2(가까운순서).csv', encoding='cp949')

In [119]:
occur_D['시도']=occur_D['주소'].map(lambda x : x.split()[0])

In [120]:
occur_D['시군구']=occur_D['주소'].map(lambda x : x.split()[1])

In [121]:
occur_D.columns[1:107]

Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       ...
       '97', '98', '99', '100', '101', '102', '103', '104', '105', '106'],
      dtype='object', length=106)

In [122]:
for i in range(len(occur_D)):
    for col in occur_D.columns[1:107]:
        occur_D.loc[i,col]=np.round(float(occur_D.loc[i,col].split()[0]),1)

In [123]:
reset_col=['시도','시군구']+(list(occur_D.columns[1:107]))

In [124]:
occur_D=occur_D[reset_col]

In [195]:
occur_D.head()

,시도,시군구,1,2,3,4,5,6,7,8,...,97,98,99,100,101,102,103,104,105,106
0,강원도,인제군,44.7,50.6,57.1,78.8,79.3,85.5,93.8,95.4,...,269.1,273.6,275.1,279.4,279.9,293.6,297.2,320.9,358.1,389.3
1,강원도,횡성군,12.5,34.9,38.9,40.9,44.7,49.9,51.2,55.3,...,224.6,230.9,231.2,235.2,237.2,249.8,254.7,278.0,313.6,369.9
2,경기도,광주시,14.3,14.6,15.3,17.6,20.6,22.6,26.2,27.5,...,244.3,244.8,244.9,245.3,251.7,263.3,264.8,265.4,274.4,289.5
3,경기도,김포시,6.4,12.0,13.2,30.5,32.7,36.3,38.4,41.1,...,271.3,273.7,300.5,301.4,303.7,303.8,311.9,319.6,322.5,323.5
4,경기도,남양주시,14.6,14.8,16.3,17.5,17.6,21.4,21.5,25.0,...,254.7,256.5,256.8,257.1,261.4,275.2,275.7,277.1,283.9,286.3


In [191]:
temp=occur_D.query('시도=="강원도" and 시군구=="횡성군"').T

In [138]:
temp.drop(['시도','시군구'], inplace=True)

In [151]:
temp=temp.astype('float')

In [168]:
temp=temp.rename(columns={0:'dist'})

In [173]:
d=50
e=60
temp.query("dist>@d and dist<@e")

,dist
2,50.6
3,57.1


In [177]:
dist_range=list(range(10,410,10))

In [218]:
sumit

,시도,시군구,month,year,day,cnt
0,강원도,인제군,1,2017,3,1
1,강원도,횡성군,1,2017,6,1
2,경기도,광주시,1,2015,19,1
3,경기도,남양주시,1,2021,11,1
4,경기도,연천군,1,2017,12,1
...,...,...,...,...,...,...
285,경기도,이천시,12,42310,241,21
286,경기도,포천시,12,46368,241,23
287,충청북도,음성군,12,48327,350,24
288,전라북도,정읍시,12,50409,341,25


In [219]:
risk_distance=pd.DataFrame(columns=['시도','시군구','month','dist_range','cnt'])
for month in range(1,13):
    try:
        temp=sumit.query('month==@month').reset_index(drop=True)
        sido=temp.iloc[-1,0]
        sigun=temp.iloc[-1,1]
        dic={}
        dic['시도']=sido
        dic['시군구']=sigun
        dic['month']=month
        dic['dist_range']=[]
        dic['cnt']=[]
        dist_range=list(range(10,int(occur_D.query('시도==@sido and 시군구==@sigun').iloc[-1,-1]+10),10))
        for i in dist_range:
            index_num=occur_D.query('시도==@sido and 시군구==@sigun').index[-1]
            temp2=occur_D.query('시도==@sido and 시군구==@sigun').T
            temp2.drop(['시도','시군구'], inplace=True)
            temp2=temp2.astype('float')
            temp2=temp2.rename(columns={index_num:'dist'})
            goal=i
            start=i-10
            dic['dist_range'].append(start)
            dic['cnt'].append(len(temp2.query("dist>=@start and dist<@goal")))
        temp3=pd.DataFrame.from_dict(dic)
        risk_distance=pd.concat([risk_distance, temp3], axis=0)
        risk_distance.reset_index(inplace=True, drop=True)
    except:
        print(month)

8


In [220]:
risk_distance.to_csv("range_distance.csv", encoding='utf-8-sig', index=False)

In [223]:
example=risk_distance.query("month==1").reset_index(drop=True)

In [230]:
example['cumulative']=0
for i in range(1,len(example)):
    example.loc[i,'cumulative']=example.loc[i-1,'cumulative']+example.loc[i,'cnt']

In [231]:
example

,시도,시군구,month,dist_range,cnt,cumulative
0,경기도,안성시,1,0,0,0
1,경기도,안성시,1,10,2,2
2,경기도,안성시,1,20,4,6
3,경기도,안성시,1,30,8,14
4,경기도,안성시,1,40,7,21
5,경기도,안성시,1,50,7,28
6,경기도,안성시,1,60,5,33
7,경기도,안성시,1,70,12,45
8,경기도,안성시,1,80,3,48
9,경기도,안성시,1,90,5,53


In [244]:
example['cumulative'].quantile(.50)

78.0

In [235]:
risk_distance2=pd.DataFrame(columns=['시도','시군구','month','dist_range','cnt'])
for month in range(1,13):
    try:
        temp=sumit.query('month==@month').reset_index(drop=True)
        sido=temp.iloc[0,0]
        sigun=temp.iloc[0,1]
        dic={}
        dic['시도']=sido
        dic['시군구']=sigun
        dic['month']=month
        dic['dist_range']=[]
        dic['cnt']=[]
        dist_range=list(range(10,int(occur_D.query('시도==@sido and 시군구==@sigun').iloc[-1,-1]+10),10))
        for i in dist_range:
            index_num=occur_D.query('시도==@sido and 시군구==@sigun').index[-1]
            temp2=occur_D.query('시도==@sido and 시군구==@sigun').T
            temp2.drop(['시도','시군구'], inplace=True)
            temp2=temp2.astype('float')
            temp2=temp2.rename(columns={index_num:'dist'})
            goal=i
            start=i-10
            dic['dist_range'].append(start)
            dic['cnt'].append(len(temp2.query("dist>=@start and dist<@goal")))
        temp3=pd.DataFrame.from_dict(dic)
        risk_distance2=pd.concat([risk_distance2, temp3], axis=0)
        risk_distance2.reset_index(inplace=True, drop=True)
    except:
        print(month)

8


In [237]:
example2=risk_distance2.query("month==1").reset_index(drop=True)

In [241]:
example2['cumulative']=0
for i in range(1,len(example2)):
    example2.loc[i,'cumulative']=example2.loc[i-1,'cumulative']+example2.loc[i,'cnt']

In [242]:
example2

,시도,시군구,month,dist_range,cnt,cumulative
0,강원도,인제군,1,0,0,0
1,강원도,인제군,1,10,0,0
2,강원도,인제군,1,20,0,0
3,강원도,인제군,1,30,0,0
4,강원도,인제군,1,40,1,1
5,강원도,인제군,1,50,2,3
6,강원도,인제군,1,60,0,3
7,강원도,인제군,1,70,2,5
8,강원도,인제군,1,80,1,6
9,강원도,인제군,1,90,4,10


In [243]:
example2['cumulative'].quantile(.25)

12.0